In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = open('Desktop/Text_For_Generation.txt', 'r')
text = path_to_file.read()

In [3]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

99 unique characters


In [4]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [5]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n Вчера был с' ---- characters mapped to int ---- > [ 0  1 31 83 65 76 60  1 61 87 71  1 77]


In [6]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])



 
В
ч
е


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\n Вчера был самый тупой диалог в моей жизни -Откройте сумку - -Это что? -Это мешки для пылесоса -Нахр'
'ена тебе на концерте мешки для пылесоса? -А Нахрена тебе знать зачем мне на концерте мешки для пылесо'
'са? - - -Проходи Гыгыгы А действительно нахрена те на концерте мешки для пылесоса?\n \nПрикинь - написа'
'л провокационный коммент "люди-вегетарианцы зачастую отстают в умственном развитии от мясоедов"Получи'
'л ответ "Вы бредете Гарила такая худая она же бедная мясо ниестТакие люди как Давинчи Пефагор повашем'


In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n Вчера был самый тупой диалог в моей жизни -Откройте сумку - -Это что? -Это мешки для пылесоса -Нах'
Target data: ' Вчера был самый тупой диалог в моей жизни -Откройте сумку - -Это что? -Это мешки для пылесоса -Нахр'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 1 (' ')
Step    2
  input: 1 (' ')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 1 (' ')


In [10]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 99) # (batch_size, sequence_length, vocab_size)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           25344     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 99)            101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


In [16]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [17]:
sampled_indices

array([30, 10, 82, 69,  2, 21, 10, 96, 63, 54, 91, 10, 25, 81, 93, 38, 87,
       61, 47,  8, 41, 28, 73,  4, 12, 37,  4,  1, 51, 80, 56, 13, 11, 79,
       28, 87, 47, 57, 25, 83, 56, 69, 54, 65, 31, 17, 38,  1,  0, 20, 36,
       25, 72, 13, 40, 47,  6, 47, 17, 89, 72, 81, 35, 26, 16, 63, 86, 56,
       50, 28, 59, 80, 74, 28,  0, 11, 94, 66, 46, 79, 14, 38, 91, 48, 44,
       29, 28, 27, 18, 19,  4, 25, 77, 96, 57, 66, 90, 96, 61, 36],
      dtype=int64)

In [18]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'н лежит у нее на столе  в пентограмме ссо скочем на рту пакетом на голове обмотан кровавым бинтом во'

Next Char Predictions: 
 "Б-цй!|-…гЩя-«х–ЙыбТ'МЁн#;И# ЦфЬ=/уЁыТЭ«чЬйЩеВ_Й \n{З«м=ЛТ%Т_эмхЖ\xad^гъЬХЁЯфоЁ\n/—жСу?ЙяУПАЁ»`f#«с…Эжю…бЗ"


In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 99)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.593661


In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
EPOCHS = 10

In [23]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
28/28 [==============================] - 86s 3s/step - loss: 4.2348
Epoch 2/10
28/28 [==============================] - 86s 3s/step - loss: 3.1102
Epoch 3/10
28/28 [==============================] - 87s 3s/step - loss: 2.8453
Epoch 4/10
28/28 [==============================] - 92s 3s/step - loss: 2.7415
Epoch 5/10
28/28 [==============================] - 88s 3s/step - loss: 2.6760
Epoch 6/10
28/28 [==============================] - 89s 3s/step - loss: 2.6207
Epoch 7/10
28/28 [==============================] - 88s 3s/step - loss: 2.5690
Epoch 8/10
28/28 [==============================] - 88s 3s/step - loss: 2.5165
Epoch 9/10
28/28 [==============================] - 88s 3s/step - loss: 2.4657
Epoch 10/10
28/28 [==============================] - 97s 3s/step - loss: 2.4169


In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            25344     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 99)             101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


In [33]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [41]:
TeXt = generate_text(model, start_string=U"Новый баш")


In [42]:
print(TeXt)

Новый башке вынили язном подторитсцони оде"а довоегу бриздеговатиПрэибрана тедегу ру простое-Себеля куй за чиску бузнойсараОДА!ТОГЮ ДЕ РА м дея завалине балишь!
 
f    f У тяжца твенний покомрат идиниет лю устуюда в мне ит рас-кум идиль это мо приссовел не пошезня адци схдри стокуть поппит на и это иденнезно лиши Заххомий скрышке отирый ожи недечаю лажа сегта ьезмы ни бель? -Том я елижал не в зигоне подевкум (сербукун!
 
f Загныечае сетер тевида ры зногда тек ожвый не беретслитенис напаеть 
 

 
(
 
 За тобу на ла леби напане порно от бедкить с собонтаваепся и иверну и начие на моминрей подалутам разна голан молати выйскутоворов он?
 
Хозныя тарева-тол бьзноса
 
 Просте ночерно вы мат"я инитьце в оглачноты эмоне не за изине?? На двуже игдем вылает моередник в очегой дивамил с рыповузмы Ихатилеви и завалие продлу!
 
fНаю гаровне о Ещеем ?Хучкейбот Онного сколенкиндерю чтоко си бель шебуть ранитойтово оф"=удевчал?и дуйлка что это тах плю
 
 "ОНо ДЛАСЬНЬ  ВСЖЕЧТЬДЬЛЬ о в кок извони спозну